In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
# relative imports
from uncertainty_aware_diagnosis import (
    ICD10data,
    split_and_save_csv,
    SimpleMLP,
    sklearnMLP,
)

# absolute imports
import polars as pl
from torch.utils.data import DataLoader
from torch import nn
import torch
import numpy as np
from torchmetrics import F1Score, Recall, Accuracy

# paths
train_csv = "./data/lbz-train.csv"
val_csv = "./data/lbz-val.csv"
test_csv = "./data/lbz-test.csv"
input_csv = "B:/Uitwissel DV en IP/Mike/Bestanden voor Luuk/Opnamen Luuk.csv"

In [14]:
opnamen = pl.read_csv(input_csv)
opnamen.head()

FileNotFoundError: No such file or directory (os error 2): B:/Uitwissel DV en IP/Mike/Bestanden voor Luuk/Opnamen Luuk.csv

In [ ]:
# split data into train, val, test
target = "icd10_main_code"

split_and_save_csv(
    input_csv=input_csv,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=test_csv,
    train_frac=0.7,
    val_frac=0.15,
    test_frac=0.15,
    stratify_col=target,  # preserve class balance
    random_state=42,
)

FileNotFoundError: No such file or directory (os error 2): ./data/corrupted_synthetic_data.csv